This code import all datasets to create a single dataset with all the information. The code is very simple and the only thing that it does is to import all the datasets and then merge them into a single dataset.

In [4]:
# Importing the libraries
import pandas as pd
import numpy as np

In [40]:
# Let's import the data

file_path_macro = '../data/df_macroeco.csv'
file_path_gdp = '../data/df_gdp_rate.csv'
file_path_inflation = '../data/df_inflation.csv'
file_path_interest = '../data/df_interest_rate.csv'
file_path_commodities = '../data/df_commodities.csv'
file_path_budget = '../data/df_budget.csv'
file_path_focus = '../data/df_FOCUS.csv'
file_path_currencies = '../data/df_currencies.csv'
file_path_revenue = '../data/df_revenue_FBCF.csv'
file_path_ibov = '../data/IBOV_historical.csv'

df_macro_monthly = pd.read_csv(file_path_macro, parse_dates=['Date'], index_col='Date') # DF with brazilian macro data
df_gdp_monthly = pd.read_csv(file_path_gdp, parse_dates=['Date'], index_col='Date') # DF with GDP data from other countries
df_inflation_rotated_monthly = pd.read_csv(file_path_inflation, parse_dates=['Date'], index_col='Date') # DF with inflation data from other countries
df_interest_rotated_monthly = pd.read_csv(file_path_interest, parse_dates=['Date'], index_col='Date') # DF with interest rate data from other countries
df_commodities = pd.read_csv(file_path_commodities, parse_dates=['Date'], index_col='Date') # DF with commodities historical prices
df_budget = pd.read_csv(file_path_budget, parse_dates=['Date'], index_col='Date') # DF with brazilian budget data
df_focus_indicators_monthly = pd.read_csv(file_path_focus, parse_dates=['Date'], index_col='Date')# DF with market expectations indicators from Brazilian Central Bank
df_currencies_monthly = pd.read_csv(file_path_currencies, parse_dates=['Date'], index_col='Date')# DF with currencies historical prices - USD, EUR and CHN
df_revenue =  pd.read_csv(file_path_revenue, parse_dates=['Date'], index_col='Date')
df_ibov =  pd.read_csv(file_path_ibov, parse_dates=['Date'], index_col='Date')

In [ ]:
# Use this to import the data from py scripts

# from import_budget_data import df_budget
# from import_oecd_api import df_gdp_monthly, df_inflation_rotated_monthly, df_interest_rotated_monthly
# from import_bcb_api import df_macro_monthly
# from import_bcb_currencies import df_currencies_monthly
# from import_bcb_focus import df_focus_indicators_monthly
# from import_imf_api import df_commodities
# from import_ipea_api import df_revenue

In [41]:
dataframes = [df_macro_monthly, # DF with brazilian macro data
              df_gdp_monthly, # DF with GDP data from other countries
              df_inflation_rotated_monthly, # DF with inflation data from other countries
              df_interest_rotated_monthly, # DF with interest rate data from other countries
              df_commodities, # DF with commodities historical prices
              df_budget, # DF with brazilian budget data
              df_focus_indicators_monthly, # DF with market expectations indicators from Brazilian Central Bank
              df_currencies_monthly,# DF with currencies historical prices - USD, EUR and CHN
              df_revenue,# DF with brazilian total revenue for federal government 
              df_ibov] # DF with IBOV historical prices

In [42]:
# Create a new DF with all dataframes merged
merged_df = df_macro_monthly # the merged df will start with the dataframe which contans the target variable

#loop to merge dataframes
for df in dataframes:
    df.rename_axis('Date', inplace=True) #rename index
    if df is not df_macro_monthly:
        merged_df = pd.merge(merged_df, df, on='Date', how='left') #merge dataframes

In [53]:
# Coverting all columns to float
#cleanned_df = merged_df.replace('', np.nan).copy() #replace empty cells with NaN
#cleanned_df = merged_df.astype(float)

cleanned_df = merged_df.astype(float).copy()

# Fill NaN values interpollating the values of the column
cleanned_df.interpolate(limit_area='inside', inplace = True)

# Fill NaN values in the beggining of each time series with the next value of the column
cleanned_df.bfill(limit=None, inplace = True)

# Drop rows which index is bigger than 2024-01-01. 
cleanned_df = cleanned_df[cleanned_df.index < '2024-01-01']

# Fill NaN values in the beggining of each time series with the next value of the column
cleanned_df.ffill(limit=None, inplace = True)



In [55]:
# Drop some columns, due to lack of data
#cleanned_df.drop(columns=['exp_gross_public_debt_y+1', 'exp_gross_public_debt_y', 'exp_gross_public_debt_y+2', 'ARG_inflation', 'eco_gross_debt_gdp_pos', 'eco_gross_debt_R$_pos'], inplace=True)


# Check for NaN values
#checks_df = merged_df.info()
nan_counts = cleanned_df.isna().sum()
print(nan_counts.sort_values(ascending=False)) #print NaN counts for each column

nan_counts.sort_values(ascending=False).to_csv('nan_counts.txt', header=False)

ECO_fiscal_result_month          0
BUDGET_fun_12_spent_value        0
EXPEC_basic_interest_rate_y+2    0
EXPEC_basic_interest_rate_y+1    0
EXPEC_basic_interest_rate_y      0
                                ..
WD_US_inflation                  0
WD_UK_inflation                  0
WD_EUZ_inflation                 0
WD_CHN_inflation                 0
IBOV                             0
Length: 81, dtype: int64


In [56]:
# Export merged df to csv
cleanned_df.to_csv('../data/cleanned_df.csv')